In [ ]:
%load_ext autoreload
%autoreload 2

## Non-std libs
import pandas as pd
import requests

## Local libs
from scn_rrd import rrd_utils

### Retrieve metadata about devices and ports, via LibreNMS's api.

Essentially we want to dump a portion of LibreNMS's mysql into pandas DataFrames.

In [ ]:
api_url_pfx = f"http://{rrd_utils.DOTENV_ENTRIES['NMS_HOST_NAME']}/api/v0"
api_headers = {
    'Authorization': f"Bearer {rrd_utils.DOTENV_ENTRIES['LIBRENMS_API_TOKEN']}"
}
def req_librenms(api_url_sfx: str):
    resp = requests.get(f'{api_url_pfx}{api_url_sfx}', headers=api_headers)
    assert resp.status_code == 200
    return resp.json()

devices = req_librenms('/devices')['devices']
devices_df = pd.DataFrame(devices)

ports = req_librenms('/ports?columns=device_id,port_id')['ports']
ports_df = pd.DataFrame(ports)

portGroups = req_librenms('/port_groups')['groups']
portGroups_df = pd.DataFrame(portGroups)
portGroups_df.rename(columns={'id': 'port_group_id'}, inplace=True)

portGroups_ports = []
for portGrp in portGroups:
    portGrpId = portGrp['id']
    _ports = req_librenms(f"/port_groups/{portGrpId}")['ports']
    for _port in _ports:
        _port['port_group_id'] = portGrpId
    portGroups_ports.extend(_ports)
    # To avoid this n+1 querying, we could `select * from librenms.port_group_port;`.
portGroups_ports_df = pd.DataFrame(portGroups_ports)

### Extract relationships out of metadata.

In [ ]:
devices_ports_portGrps_df = (
    devices_df[['device_id', 'hostname', 'location']] 
    .merge(ports_df, on='device_id')
    .merge(portGroups_ports_df, on='port_id', how='left')
    .merge(
        portGroups_df.drop(columns='desc').rename(columns={'name': 'port_group_name'}),
        on='port_group_id',
        how='left')
)
devices_ports_portGrps_df['port_rrd_filename'] = [
    f"port-id{row['port_id']}.rrd"
    for (_, row) in devices_ports_portGrps_df.iterrows()
]

In [ ]:
rrd_utils.write_devices_ports(devices_ports_portGrps_df)

In [ ]:
rrd_utils.read_devices_ports()